In [94]:
import pandas as pd
import numpy as np
import matplotlib as plt
import sklearn
import seaborn as sns

In [95]:
df = pd.read_csv("./data/bank-full.csv")
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no


In [96]:
has_null = df.isnull().any().any()
has_null

False

In [97]:
def encode_categorical(df_orginal):
    df = df_orginal.copy(deep=True)
    
    months = ['jan', 'feb',
       'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
    month_num = {months[i]: i+1 for i in range(len(months))}
    df["month"] = df["month"].map(month_num)
    
    edu_num = {'tertiary':2, 'secondary':1, 'unknown':-1, 'primary':0}
    df["education"] = df["education"].map(edu_num)
    
    marital_num = {'single':0, 'married':1, 'divorced':2}
    df["marital"] = df["marital"].map(marital_num)
    
    # Encoding for binary columns
    binary_num = {"no":0 ,"yes":1}
    binaries = [col for col in df.columns if set(df[col].unique()) == {"yes", "no"}]
    df[binaries] = df[binaries].apply(lambda x: x.map(binary_num))
    
    # One Hot Encoding for specified columns
    for column in ['contact', 'job', 'poutcome']:
        df = df.join(pd.get_dummies(df[column], prefix=column))
        del df[column]
        
    df.columns = df.columns.str.replace('.', ' ')
    
    non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
    if len(non_numeric_cols) == 0:
        print("All columns numeric")
    else:
        print("Non-numeric columns are: ", non_numeric_cols)
    
    return df

In [98]:
df_changed = encode_categorical(df)

All columns numeric


C:\Users\Wojtek\AppData\Local\Temp\ipykernel_9736\601411229.py:25: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns = df.columns.str.replace('.', ' ')


In [99]:
df_changed.head()

,age,marital,education,default,balance,housing,loan,day,month,duration,...,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,58,1,2,0,2143,1,0,5,5,261,...,0,0,0,0,0,0,0,0,0,1
1,44,0,1,0,29,1,0,5,5,151,...,0,0,0,1,0,0,0,0,0,1
2,33,1,1,0,2,1,1,5,5,76,...,0,0,0,0,0,0,0,0,0,1
3,47,1,-1,0,1506,1,0,5,5,92,...,0,0,0,0,0,0,0,0,0,1
4,33,0,-1,0,1,0,0,5,5,198,...,0,0,0,0,0,1,0,0,0,1
